# Predict stock prices  with Long short-term memory (LSTM)

This simple example will show you how LSTM models predict time series data. Stock market data is a great choice for this because it's quite regular and widely available via the Internet. 

## Install requirements
We install Tensorflow 2.0 with GPU support first

In [ ]:
!pip install -qU pip
!pip install -q tensorflow pandas-datareader

In [ ]:
!pip install -q pydot pydot-ng

In [ ]:
!pip install -Uq yfinance pandas-datareader

In [ ]:
import os

scratch_path = os.environ.get("SCRATCH", "../scratch")
if not os.path.exists(scratch_path):
    os.makedirs(scratch_path)

## Introduction

LSTMs are very powerful in sequence prediction problems. They can store past information.

## Loading the dataset
I use pandas-datareader to get the historical stock prices from Yahoo! finance. For this example, I get only the historical data till the end of *training_end_data*.  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
import numpy as np  # python library for scientific computing
import pandas as pd  # python library for data manipulation and analysis
import matplotlib.pyplot as plt  # python library for charting
from pandas_datareader import (
    data as pdr,
)  # extract data from internet sources into pandas data frame
import yfinance as yf
import tensorflow as tf

In [ ]:
tickers = "IBM"

start_date = "1980-12-01"
end_date = "2018-12-31"

yf.pdr_override()
stock_data = pdr.get_data_yahoo(tickers, start_date)

In [ ]:
stock_data_len = stock_data["Close"].count()
print(f"Read in {stock_data_len} stock values")

In [ ]:
close_prices = stock_data.iloc[:, 1:2].values
# print(close_prices)

Of course, some of the weekdays might be public holidays in which case no price will be available. For this reason, we will fill the missing prices with the latest available prices

In [ ]:
all_bussinessdays = pd.date_range(start=start_date, end=end_date, freq="B")
print(all_bussinessdays)

In [ ]:
close_prices = stock_data.reindex(all_bussinessdays)
close_prices = stock_data.fillna(method="ffill")

The dataset is now complete and free of missing values. Let's have a look to the data frame summary:

## Feature scaling

In [ ]:
training_set = close_prices.iloc[:, 1:2].values

In [ ]:
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler(feature_range=(0, 1))
training_set_scaled = sc.fit_transform(training_set)
# print(training_set_scaled.shape)

LSTMs expect the data in a specific format, usually a 3D tensor. I start by creating data with 60 days and converting it into an array using NumPy. Next, I convert the data into a 3D dimension array with feature_set samples, 60 days and one feature at each step.

In [ ]:
features = []
labels = []
for i in range(60, stock_data_len):
    features.append(training_set_scaled[i - 60 : i, 0])
    labels.append(training_set_scaled[i, 0])

features = np.array(features)
labels = np.array(labels)

features = np.reshape(features, (features.shape[0], features.shape[1], 1))

Feature tensor with three dimension: features[0] contains the ..., features[1] contains the last 60 days of values and features [2] contains the  ...

## Create the LSTM network
Let's create a sequenced LSTM network with 50 units. Also the net includes some dropout layers with 0.2 which means that 20% of the neurons will be dropped.

In [ ]:
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.LSTM(
            units=50, return_sequences=True, input_shape=(features.shape[1], 1)
        ),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.LSTM(units=50, return_sequences=True),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.LSTM(units=50, return_sequences=True),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.LSTM(units=50),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(units=1),
    ]
)

In [ ]:
print(model.summary())

The model will be compiled and optimize by the adam optimizer and set the loss function as mean_squarred_error

In [ ]:
model.compile(optimizer="adam", loss="mean_squared_error")

In [ ]:
from time import time

start = time()
history = model.fit(features, labels, epochs=5, batch_size=32, verbose=1)
end = time()

In [ ]:
print("Total training time {} seconds".format(end - start))

In [ ]:
import os
# save tf model format
tf.keras.models.save_model(model, os.path.join(scratch_path, "stocks/1"))

In [ ]:
import tf2onnx
import onnx
import os

# onnx model format
model_proto, _ = tf2onnx.convert.from_keras(model)
onnx.save(model_proto, os.path.join(scratch_path, "stocks.onnx"))

In [ ]:
testing_start_date = "2019-01-01"
testing_end_date = "2019-04-10"

test_stock_data = pdr.get_data_yahoo(tickers, testing_start_date, testing_end_date)

In [ ]:
test_stock_data_processed = test_stock_data.iloc[:, 1:2].values

In [ ]:
all_stock_data = pd.concat((stock_data["Close"], test_stock_data["Close"]), axis=0)

In [ ]:
inputs = all_stock_data[len(all_stock_data) - len(test_stock_data) - 60 :].values
inputs = inputs.reshape(-1, 1)
inputs = sc.transform(inputs)

In [ ]:
X_test = []
for i in range(60, 129):
    X_test.append(inputs[i - 60 : i, 0])

In [ ]:
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_stock_price = model.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(
    test_stock_data_processed, color="blue", label="Actual " + tickers + "Stock Price"
)
plt.plot(
    predicted_stock_price, color="red", label="Predicted " + tickers + "Stock Price"
)
plt.title("Actual " + tickers + "Stock Price Prediction")
plt.xlabel("Date")
plt.ylabel(tickers + " Stock Price")
plt.legend()
plt.show()